In [22]:
import polars as pl
import pandas as pd
import numpy as np
import os, glob
import re
import plotly.express as px
import polars as pl

In [23]:
# directories
cwd = os.getcwd() # directory this file is saved in
data_dir = os.path.join(cwd, "BCDC-Metadata") # BCDC data folder
metadata_dir = os.path.join(data_dir, 'Sample-Inventory') # directory with specimen metadata
project_dir = os.path.join(data_dir, 'Data-Collection-Inventory', '2022Q3_csvs')

df = pd.read_csv(os.path.join(cwd, 'BCDC_Metadata_2022Q3_v7.csv'), encoding='unicode_escape')
df.columns = df.columns.str.replace(r"\([^)]*\)","").str.strip()
df_zeng = pd.read_csv(os.path.join(cwd, 'BCDC_Metadata_U19_Zeng_SSv4_Sample_Inventory.csv'), encoding='unicode_escape')
df_zeng.columns = df_zeng.columns.str.replace(r"\([^)]*\)","").str.strip()
df = pd.concat([df, df_zeng]).reset_index(drop=True)

### use a data cleaning function here that takes both this dataset and CV
# 1. get rid of all the special characters in CV columns and column names
# 2. cv and data columns both to lower case
# 3. way to identify extremely similar words (i.e. brain vs brains)
### in simple code logic...
# list of cols with controlled values
controlled_cols = [re.sub("\([^)]*\)", "", x).strip() for x in list(filter(lambda x : 'CV' in x, df.columns))]
#df.columns = df.columns.str.replace(r"\([^)]*\)","").str.strip() # get rid of (CV) in data columns
#print(df['Modality'])
df['Sample Type'] = df['Sample Type'].str.replace("[^A-Za-z0-9 ]+", " ").str.lower() # special characters into spaces
# to-do: do the same for all CV columns
df['Subspecimen Type'] = df['Subspecimen Type'].str.replace("[^A-Za-z0-9 ]+", " ").str.lower() # special characters into spaces
df['Total Processed Subspecimens'] = df['Total Processed Subspecimens'].astype(str).str.replace(',', '').astype(float) # cell counts is numeric
df['Sample ID'] = df['Sample ID'].astype(str)
df['Species'] = df['Species'].str.lower()
df['Modality'] = df['Modality'].str.lower()
df['Technique'] = df['Technique'].str.lower()
df['R24 Name'] = df['R24 Name'].str.upper()

df['Sample Type'] = df['Sample Type'].fillna(df['Subspecimen Type'])
df['Subspecimen Type'] = df['Subspecimen Type'].fillna(df['Sample Type'])
df['Total Processed Subspecimens'] = df['Total Processed Subspecimens'].fillna(0)
### collect systematic patterns for auto-correction
optional_cols = ['Age', 'Sex', 'Genotype', 'Species NCBI Taxonomy ID']
df = df.drop(optional_cols, axis = 1)
df = df.drop_duplicates()

df.loc[(df['Technique'].str.contains('mc-seq', case = False)) & (~df['Technique'].str.contains('sn', case = False)), 'Technique'] = 'mC-seq2'
df.loc[(df['Technique'].str.contains('10x Chromium 3', case = False)) & (~df['Technique'].str.contains(';', case = False)), 'Technique'] = '10x chromium 3\' sequencing'


df

/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_72257/3040083465.py:7: DtypeWarning: Columns (4,14,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(cwd, 'BCDC_Metadata_2022Q3_v7.csv'), encoding='unicode_escape')
/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_72257/3040083465.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(r"\([^)]*\)","").str.strip()
/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_72257/3040083465.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_zeng = pd.read_csv(os.path.join(cwd, 'BCDC_Metadata_U19_Zeng_SSv4_Sample_Inventory.csv'), encoding='unicode_escape')
/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_72257/3040083465.py:10: FutureWarning: The default value of regex will change from True to False in a future v

,Sample ID,Sample Type,Species,Parent Specimen ID,Parent Specimen Type,Subject ID,Modality,Technique,Anatomical Structure,Subspecimen Type,...,Organization,Investigator,Grant Number,Data Collection,R24 Name,R24 link,Comments,Metadata Submission,Anatomical Substructure,R24.link
0,SW170829-01A,whole brain,mouse,NaN,NaN,SW170829-01A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
1,SW170829-02A,whole brain,mouse,NaN,NaN,SW170829-02A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
2,SW170910-01A,whole brain,mouse,NaN,NaN,SW170910-01A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
3,SW170917-01A,whole brain,mouse,NaN,NaN,SW170917-01A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
4,SW171101-01A,whole brain,mouse,NaN,NaN,SW171101-01A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305889,L8S4_180620_01_H09,whole cell,mouse,NaN,NaN,393684.0,transcriptomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305890,L8S4_180620_01_A11,whole cell,mouse,NaN,NaN,393684.0,transcriptomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305891,L8S4_180620_01_H11,whole cell,mouse,NaN,NaN,393684.0,transcriptomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305892,L8S4_180129_04_H11,whole cell,mouse,NaN,NaN,366287.0,transcriptomics,smart-seq v4,RSP,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,RSP,NaN


In [24]:
### triage the multimodal/multi technique ones
### i.e. the ones with semicolumns or parenthesis
### approximately 5000 rows
df = df.loc[(~df['Technique'].str.contains(';|\(')) & (~df['Modality'].str.contains(';'))]

In [25]:
df

,Sample ID,Sample Type,Species,Parent Specimen ID,Parent Specimen Type,Subject ID,Modality,Technique,Anatomical Structure,Subspecimen Type,...,Organization,Investigator,Grant Number,Data Collection,R24 Name,R24 link,Comments,Metadata Submission,Anatomical Substructure,R24.link
0,SW170829-01A,whole brain,mouse,NaN,NaN,SW170829-01A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
1,SW170829-02A,whole brain,mouse,NaN,NaN,SW170829-02A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
2,SW170910-01A,whole brain,mouse,NaN,NaN,SW170910-01A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
3,SW170917-01A,whole brain,mouse,NaN,NaN,SW170917-01A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
4,SW171101-01A,whole brain,mouse,NaN,NaN,SW171101-01A,connectivity,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305889,L8S4_180620_01_H09,whole cell,mouse,NaN,NaN,393684.0,transcriptomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305890,L8S4_180620_01_A11,whole cell,mouse,NaN,NaN,393684.0,transcriptomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305891,L8S4_180620_01_H11,whole cell,mouse,NaN,NaN,393684.0,transcriptomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305892,L8S4_180129_04_H11,whole cell,mouse,NaN,NaN,366287.0,transcriptomics,smart-seq v4,RSP,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,RSP,NaN


In [26]:
### what modality each technique should belong
## coerce them, all lowercase
modality_tech_map = {
    'multimodal': ['whole cell patch clamp', 'patch-seq', 'share-seq', '10x genomics multiome', '10x chromium multiome'], 
    'epigenomics': ['atac-seq', 'sci-atac-seq3', 'mC-seq2'], 
    'anatomy/morphology' : ['cre-dependent anterograde tracing', 'anterograde tracing', 'retrograde tracing', 
                            'retrograde transsynaptic tracing', 
                            'enhancer virus labeling', 'morf genetic sparse labeling', 'histology', 'trio tracing', 
                           'stereology', 'triple anterograde', 'olst'], # triple?
    'cell morphology' : ['neuron morphology reconstruction'],
    "imaging" : ['confrocal microscopy', 'confocal microscopy', 'stpt', 'pct', 'oct', "lightsheet microscopy", 
                 "light sheet microscopy", "mouselight", 'mri', 'cisi', 'fmost',
                 'visor'], # typo here
    "spatial transcriptomics" : ['fish', 'seqfish', 'merfish', 'slide-seq', 'smfish'],
    'transcriptomics/epigenomics' : ['10x chromium 3\' sequencing', 'smart-seq v4', 'sci-rna-seq3', 'drop-seq', 'dnaseq', 
                         'pacbio long read sequencing']
    
}

## multiomics - 10x genetics multiome, share-seq

## techniques that do not exist in paper
## 10x chromium multiome (multimodal), stereology (anatomy/morphology), mC-seq2 (Epigenetics)
## triple anterograde (anatomy/morphology), olst, smfish

df['Modality'] = df['Technique'].apply(lambda i:[k for k, v in modality_tech_map.items() if i in v]).str[0] # should just have 1 for now
# calculate new sum in collapsed group
df


#### plot for each modality
## each modality is what the data for 

/var/folders/76/jdv5bsqj4_9dk0rm_g_vp9gm0000gp/T/ipykernel_72257/2107260094.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Modality'] = df['Technique'].apply(lambda i:[k for k, v in modality_tech_map.items() if i in v]).str[0] # should just have 1 for now


,Sample ID,Sample Type,Species,Parent Specimen ID,Parent Specimen Type,Subject ID,Modality,Technique,Anatomical Structure,Subspecimen Type,...,Organization,Investigator,Grant Number,Data Collection,R24 Name,R24 link,Comments,Metadata Submission,Anatomical Substructure,R24.link
0,SW170829-01A,whole brain,mouse,NaN,NaN,SW170829-01A,anatomy/morphology,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
1,SW170829-02A,whole brain,mouse,NaN,NaN,SW170829-02A,anatomy/morphology,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
2,SW170910-01A,whole brain,mouse,NaN,NaN,SW170910-01A,anatomy/morphology,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
3,SW170917-01A,whole brain,mouse,NaN,NaN,SW170917-01A,anatomy/morphology,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
4,SW171101-01A,whole brain,mouse,NaN,NaN,SW171101-01A,anatomy/morphology,anterograde tracing,NaN,whole brain,...,University of Southern California,Hong-Wei Dong,1U19MH114821-01,huang_antero,BIL,http://download.brainimagelibrary.org/biccn/hu...,NaN,2018Q3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305889,L8S4_180620_01_H09,whole cell,mouse,NaN,NaN,393684.0,transcriptomics/epigenomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305890,L8S4_180620_01_A11,whole cell,mouse,NaN,NaN,393684.0,transcriptomics/epigenomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305891,L8S4_180620_01_H11,whole cell,mouse,NaN,NaN,393684.0,transcriptomics/epigenomics,smart-seq v4,PAL,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,PAL - GP,NaN
305892,L8S4_180129_04_H11,whole cell,mouse,NaN,NaN,366287.0,transcriptomics/epigenomics,smart-seq v4,RSP,whole cell,...,Allen Institute for Brain Science,Zeng,1U19MH114830-01,zeng_sc_SSv4,NEMO,NaN,NaN,2022Q2,RSP,NaN


In [27]:
# grant/data collection lookup table
key_df = pd.read_csv(os.path.join(project_dir, 'data_collection_is_specified_output_of_data_collection_project.csv')).drop('priority_order', axis = 1)
# grant/data collection lookup table
grant_df = pd.read_csv(os.path.join(project_dir, 'grant_is_specified_input_of_data_collection_project.csv')).drop('priority_order', axis = 1)
metadata_df = grant_df.merge(key_df, on = 'project_reference_id', how = 'outer')[['data_collection_reference_id', 'grant_reference_id']]

#### Count for number of samples

In [28]:
## counts on sample type level
sample_count_df = pd.DataFrame(df.groupby(['Data Collection', 'Sample Type', 'Metadata Submission', 'Species', 'Modality', 'Technique', 'R24 Name'], as_index = False).agg({'Sample ID': 'nunique'})).rename(columns = {'Data Collection':'data_collection_reference_id', 'Sample ID':'sample_count', 'Metadata Submission':'quarters', 'Sample Type': 'sample_name', 'Species':'species', 'Modality':'modality', 'Technique':'technique', 'R24 Name':'archive', 'Sample Type':'sample_type'})
## count the unique instances of sample IDs and subject IDs for each unique combination of project, quarter and sample type
sample_count_df

,data_collection_reference_id,sample_type,quarters,species,modality,technique,archive,sample_count
0,chung_lsfm,brain hemisphere,2021Q4,human,imaging,light sheet microscopy,DANDI,1
1,chung_lsfm,brain hemisphere,2022Q2,human,imaging,light sheet microscopy,DANDI,1
2,chung_lsfm,brain hemisphere,2022Q3,human,imaging,light sheet microscopy,DANDI,1
3,dev_mouse_brain_lsfm,whole brain,2021Q3,mouse,imaging,light sheet microscopy,BIL,117
4,dev_mouse_brain_lsfm,whole brain,2021Q4,mouse,imaging,light sheet microscopy,BIL,81
...,...,...,...,...,...,...,...,...
407,zhuang_dulac_merfish_MOE,brain region,2021Q1,mouse,spatial transcriptomics,merfish,BIL,1
408,zhuang_dulac_merfish_PAG,brain region,2020Q2,mouse,spatial transcriptomics,merfish,BIL,1
409,zhuang_dulac_merfish_PAG_2,brain region,2021Q2,mouse,spatial transcriptomics,merfish,BIL,1
410,zhuang_merfish_M1,brain region,2020Q1,mouse,spatial transcriptomics,merfish,BIL,1


In [300]:
df['Technique'].unique()

array(['anterograde tracing', "10x chromium 3' sequencing", 'stpt',
       'retrograde tracing', 'trio tracing',
       'cre-dependent anterograde tracing', 'drop-seq',
       'morf genetic sparse labeling', 'smart-seq v4', 'atac-seq',
       'mC-seq2', 'retrograde transsynaptic tracing', 'fmost',
       'mouselight', 'snare-seq2;atac-seq', 'snare-seq2;rnaseq',
       'merfish', 'fish', 'neuron morphology reconstruction',
       'patch-seq;whole cell patch clamp',
       'patch-seq;neuron morphology reconstruction',
       'patch-seq;smart-seq v4', 'seqfish', 'dnaseq',
       'stpt;enhancer virus labeling', 'visor;light sheet microscopy',
       'histology', 'mri', 'oct', 'light sheet microscopy', 'patch-seq',
       'slide-seq', 'share-seq', 'sci-rna-seq3', 'triple anterograde',
       'snare-seq2 (rna-seq)', 'snare-seq2 (atac-seq)',
       '10x genomics multiome', 'pacbio long read sequencing',
       '10x genomics multiome; atac-seq', '10x genomics multiome; rnaseq',
       'sci-ata

In [268]:
## or do this instead of coercing
#mod_map = {
#    'multimodal' : ['multimodal', 'electrophysiology'],
#    'imaging' : ['population imaging', 'histology imaging', 'cell type distribution'], # check for histology imaging
#    'anatomy/morphology' : ['cell morphology', 'connectivity', 'anatomy'],
#    'spatial' : ['spatial transcriptomics'],
#    'transcriptomics': ['transcriptomics', 'genomics', 'epigenomics']
#}



#sample_count_df['modality'] = sample_count_df['modality'].apply(lambda i:[k for k, v in mod_map.items() if i in v]).str[0] # should just have 1 for now
# calculate new sum in collapsed group
#sample_count_df# = sample_count_df.groupby(['data_collection_reference_id', 'quarters', 'species'], as_index = False)['sample_count'].sum()

,data_collection_reference_id,quarters,species,modality,technique,sample_count
0,chung_lsfm,2021Q4,human,NaN,light sheet microscopy,1
1,chung_lsfm,2022Q2,human,NaN,light sheet microscopy,1
2,chung_lsfm,2022Q3,human,NaN,light sheet microscopy,1
3,dev_mouse_brain_lsfm,2021Q3,mouse,NaN,light sheet microscopy,117
4,dev_mouse_brain_lsfm,2021Q4,mouse,NaN,light sheet microscopy,81
...,...,...,...,...,...,...
412,zhuang_dulac_merfish_MOE,2021Q1,mouse,NaN,merfish,1
413,zhuang_dulac_merfish_PAG,2020Q2,mouse,NaN,merfish,1
414,zhuang_dulac_merfish_PAG_2,2021Q2,mouse,NaN,merfish,1
415,zhuang_merfish_M1,2020Q1,mouse,NaN,merfish,1


In [378]:
sample_count_df

,data_collection_reference_id,grant_reference_id,sample_type,quarters,species,modality,technique,sample_count
0,chung_lsfm,U01 Chung grant,brain hemisphere,2021Q4,humans,imaging,light sheet microscopy,1
1,chung_lsfm,U01 Chung grant,brain hemisphere,2022Q2,humans,imaging,light sheet microscopy,1
2,chung_lsfm,U01 Chung grant,brain hemisphere,2022Q3,humans,imaging,light sheet microscopy,1
3,dev_mouse_brain_lsfm,RF1 Kim grant,whole brain,2021Q3,mouse,imaging,light sheet microscopy,117
4,dev_mouse_brain_lsfm,RF1 Kim grant,whole brain,2021Q4,mouse,imaging,light sheet microscopy,81
...,...,...,...,...,...,...,...,...
422,zhuang_dulac_merfish_PAG,U19 Huang grant,brain region,2020Q2,mouse,spatial transcriptomics,merfish,1
423,zhuang_dulac_merfish_PAG_2,U19 Huang grant,brain region,2021Q2,mouse,spatial transcriptomics,merfish,1
424,zhuang_merfish_M1,U19 Huang grant,brain region,2020Q1,mouse,spatial transcriptomics,merfish,1
425,zhuang_merfish_M1,U19 Zeng grant,brain region,2020Q1,mouse,spatial transcriptomics,merfish,1


In [29]:
species_map = {
    'non-human primates' : ['chimpanzee', 'small-eared galago', 'western gorilla', 'green monkey', 'pig-tailed macaque',
                 "ma's night monkey", 'rhesus macaque', 'bolivian squirrel monkey','marmoset',
       'crab-eating macaque'],
    'humans' : ['human'], ## mostly mice and humans
    'mouse' : ['mouse'],
    'other mammals': [ 'arctic ground squirrel',
    'nine-banded armadillo',
    'domestic cat',
    'domestic ferret',
    'gray short-tailed opossum',
    'pig', 'rabbit', 'norway rat', 'common tree shrew'
    ]
}


sample_count_df['species'] = sample_count_df['species'].apply(lambda i:[k for k, v in species_map.items() if i in v]).str[0] # should just have 1 for now
# calculate new sum in collapsed group
sample_count_df = sample_count_df.groupby(['data_collection_reference_id', 'sample_type', 'archive', 'quarters', 'species', 'modality', 'technique'], as_index = False)['sample_count'].sum()
sample_count_df = metadata_df.merge(sample_count_df, on = 'data_collection_reference_id', how = 'right')


In [32]:
sample_count_df.to_csv(os.path.join(cwd, 'dash_sample_count_df.csv'), index = False)

In [31]:
cwd

'/Users/christine.lin/Documents/GitHub/BCDC-Dashboards'

#### Count of subspecimens

In [391]:
df_donors = df.groupby(['Data Collection', 'Metadata Submission', 'Species', 'Modality', 'Technique', 'R24 Name'], as_index = False).agg({'Subject ID':'nunique'}).rename(columns = {'Data Collection':'data_collection_reference_id', 'Metadata Submission': 'quarters', 'Subject ID': 'donor_count', 'Species':'species', 'R24 Name':'archive'})
df_donors

,data_collection_reference_id,quarters,species,Modality,Technique,archive,donor_count
0,chung_lsfm,2021Q4,human,imaging,light sheet microscopy,DANDI,0
1,chung_lsfm,2022Q2,human,imaging,light sheet microscopy,DANDI,0
2,chung_lsfm,2022Q3,human,imaging,light sheet microscopy,DANDI,0
3,dev_mouse_brain_lsfm,2021Q3,mouse,imaging,light sheet microscopy,BIL,0
4,dev_mouse_brain_lsfm,2021Q4,mouse,imaging,light sheet microscopy,BIL,27
...,...,...,...,...,...,...,...
412,zhuang_dulac_merfish_MOE,2021Q1,mouse,spatial transcriptomics,merfish,BIL,0
413,zhuang_dulac_merfish_PAG,2020Q2,mouse,spatial transcriptomics,merfish,BIL,0
414,zhuang_dulac_merfish_PAG_2,2021Q2,mouse,spatial transcriptomics,merfish,BIL,0
415,zhuang_merfish_M1,2020Q1,mouse,spatial transcriptomics,merfish,BIL,0


In [33]:
## get number of donors per quarter
df_donors = df.groupby(['Data Collection', 'Metadata Submission', 'Species', 'Modality', 'Technique'], as_index = False).agg({'Subject ID':'nunique'}).rename(columns = {'Data Collection':'data_collection_reference_id', 'Metadata Submission': 'quarters', 'Subject ID': 'donor_count', 'Species':'species', 'Modality':'modality', 'Technique':'technique'})
## get number of processed subspeciemns for each unique sample
df_subspecimens = df.groupby(['Data Collection', 'Metadata Submission', 'Sample ID', 'Subspecimen Type', 'R24 Name'], as_index = False).agg({'Total Processed Subspecimens':'sum'}).drop('Sample ID', axis = 1).rename(columns = {'Data Collection':'data_collection_reference_id', 'Metadata Submission': 'quarters', 'Subspecimen Type': 'subspecimen_type', 'Total Processed Subspecimens': 'subspecimen_count', 'R24 Name': 'archive'})

df_donor_counts = df_donors.merge(df_subspecimens, on = ['data_collection_reference_id', 'quarters'])

df_donor_counts = metadata_df.merge(df_donor_counts, on = 'data_collection_reference_id', how = 'right')


df_donor_counts['species'] = df_donor_counts['species'].apply(lambda i:[k for k, v in species_map.items() if i in v]).str[0]
df_donor_counts

,data_collection_reference_id,grant_reference_id,quarters,species,modality,technique,donor_count,subspecimen_type,archive,subspecimen_count
0,chung_lsfm,U01 Chung grant,2021Q4,humans,imaging,light sheet microscopy,0,brain section set,DANDI,79.0
1,chung_lsfm,U01 Chung grant,2022Q2,humans,imaging,light sheet microscopy,0,brain section set,DANDI,41.0
2,chung_lsfm,U01 Chung grant,2022Q3,humans,imaging,light sheet microscopy,0,brain section set,DANDI,32.0
3,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q3,mouse,imaging,light sheet microscopy,0,whole brain,BIL,1.0
4,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q3,mouse,imaging,light sheet microscopy,0,whole brain,BIL,1.0
...,...,...,...,...,...,...,...,...,...,...
371273,zhuang_dulac_merfish_PAG,U19 Huang grant,2020Q2,mouse,spatial transcriptomics,merfish,0,whole cell,BIL,2900000.0
371274,zhuang_dulac_merfish_PAG_2,U19 Huang grant,2021Q2,mouse,spatial transcriptomics,merfish,0,whole cell,BIL,2900000.0
371275,zhuang_merfish_M1,U19 Huang grant,2020Q1,mouse,spatial transcriptomics,merfish,0,whole cell,BIL,300000.0
371276,zhuang_merfish_M1,U19 Zeng grant,2020Q1,mouse,spatial transcriptomics,merfish,0,whole cell,BIL,300000.0


In [278]:
dup_df = df_donor_counts[['data_collection_reference_id', 'grant_reference_id']].drop_duplicates()
dup_count = pd.DataFrame(dup_df.drop_duplicates().groupby('data_collection_reference_id', as_index = False)['grant_reference_id'].nunique())
dup_count[dup_count['grant_reference_id'] > 1]

,data_collection_reference_id,grant_reference_id
43,human_cortex_SS4_Open_GRU,2
44,human_cortex_SS4_restricted_GRU,2
45,human_cortex_SS4_restricted_limited,2
106,tasic_sc_SSv4,3
107,tasic_stpt,3
135,zhuang_merfish_M1,2


In [326]:
metadata_df[metadata_df['grant_reference_id'] == 'RF1 Macosko grant']

,data_collection_reference_id,grant_reference_id
115,macosko_mouse_slideseq_histologydataset,RF1 Macosko grant
116,macosko_mouse_slideseq_slideseqdataset,RF1 Macosko grant


In [34]:
## projects funded by multiple grants
dup_df = df_donor_counts[['data_collection_reference_id', 'grant_reference_id']].drop_duplicates()
dup_count = pd.DataFrame(dup_df.drop_duplicates().groupby('data_collection_reference_id', as_index = False)['grant_reference_id'].nunique())
multi_list = dup_count[dup_count['grant_reference_id'] > 1]['data_collection_reference_id']
df_donor_counts.loc[df_donor_counts['data_collection_reference_id'].isin(multi_list), 'grant_reference_id'] = 'multi-grant'

## projects reporting multiple species
dup_df = df_donor_counts[['data_collection_reference_id', 'species']].drop_duplicates()
dup_count = pd.DataFrame(dup_df.drop_duplicates().groupby('data_collection_reference_id', as_index = False)['species'].nunique())
multi_list = dup_count[dup_count['species'] > 1]['data_collection_reference_id']
df_donor_counts.loc[df_donor_counts['data_collection_reference_id'].isin(multi_list), 'species'] = 'multi-species'


In [36]:
specimen_count_map = {
    'brain_count' : ['whole brain','brain', 'brains', 'brain hemisphere'],
    'cell_count' : ['whole cell', 'cell', 'cells', 'cell body', 'cell nucleus', 'nuclei', 'reconstruction', 'cell in slice'],
    'library_count' : ['library'],
    'tissue_region_count' : ['brain region', 'brain section set', 'brain slice', 'spinal cord slice', 'tissue sample', 'cell suspension']
    }
    

count_df = pd.DataFrame()

for project in df_donor_counts['data_collection_reference_id'].unique():
    non_count_cols = ['data_collection_reference_id', 'grant_reference_id', 'quarters', 'species', 'modality', 'technique', 'donor_count', 'subspecimen_type', 'archive']
    count_df_project = pd.DataFrame(columns = [non_count_cols + list(specimen_count_map.keys())], index = [0])
    project_df = df_donor_counts[df_donor_counts['data_collection_reference_id'] == project]
    project_df = project_df.groupby(non_count_cols, as_index = False)['subspecimen_count'].sum()
   # print(project_df)
    ## each quarter
    for i in range(len(project_df)):
        project_row = project_df.iloc[i]
        row_container = pd.DataFrame(columns = [non_count_cols + list(specimen_count_map.keys())], index = [0])
        #row_container[non_count_cols] = project_row[non_count_cols].drop('subspecimen_type')
        row_container[['data_collection_reference_id', 'grant_reference_id', 'quarters', 'species', 'modality', 'technique', 'donor_count', 'archive']] = project_row[['data_collection_reference_id', 'grant_reference_id', 'quarters', 'species', 'modality', 'technique', 'donor_count', 'archive']]
        count_cols = [k for k, v in specimen_count_map.items() if project_row['subspecimen_type'] in v]
        row_container[count_cols] = project_row['subspecimen_count']
        count_df_project = pd.concat([count_df_project, row_container.fillna(0)])
    
    count_df_project = count_df_project.dropna(how = 'all')
    count_df = pd.concat([count_df, count_df_project])


count_df 

,data_collection_reference_id,grant_reference_id,quarters,species,modality,technique,donor_count,subspecimen_type,archive,brain_count,cell_count,library_count,tissue_region_count
0,chung_lsfm,U01 Chung grant,2021Q4,humans,imaging,light sheet microscopy,0,0,DANDI,0,0,0,79.0
0,chung_lsfm,U01 Chung grant,2022Q2,humans,imaging,light sheet microscopy,0,0,DANDI,0,0,0,41.0
0,chung_lsfm,U01 Chung grant,2022Q3,humans,imaging,light sheet microscopy,0,0,DANDI,0,0,0,32.0
0,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q3,mouse,imaging,light sheet microscopy,0,0,BIL,117.0,0,0,0
0,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q4,mouse,imaging,light sheet microscopy,27,0,BIL,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,zhuang_dulac_merfish_MOE,U19 Zeng grant,2021Q1,mouse,spatial transcriptomics,merfish,0,0,BIL,0,3300000.0,0,0
0,zhuang_dulac_merfish_PAG,U19 Huang grant,2020Q2,mouse,spatial transcriptomics,merfish,0,0,BIL,0,2900000.0,0,0
0,zhuang_dulac_merfish_PAG_2,U19 Huang grant,2021Q2,mouse,spatial transcriptomics,merfish,0,0,BIL,0,2900000.0,0,0
0,zhuang_merfish_M1,multi-grant,2020Q1,mouse,spatial transcriptomics,merfish,0,0,BIL,0,600000.0,0,0


In [37]:
count_df.to_csv(os.path.join(cwd, 'dash_specimen_count_df.csv'), index = False)

In [395]:
count_df

,data_collection_reference_id,grant_reference_id,quarters,species,modality,technique,donor_count,subspecimen_type,archive,brain_count,cell_count,library_count,tissue_region_count
0,chung_lsfm,U01 Chung grant,2021Q4,humans,imaging,light sheet microscopy,0,0,DANDI,0,0,0,79.0
0,chung_lsfm,U01 Chung grant,2022Q2,humans,imaging,light sheet microscopy,0,0,DANDI,0,0,0,41.0
0,chung_lsfm,U01 Chung grant,2022Q3,humans,imaging,light sheet microscopy,0,0,DANDI,0,0,0,32.0
0,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q3,mouse,imaging,light sheet microscopy,0,0,BIL,117.0,0,0,0
0,dev_mouse_brain_lsfm,RF1 Kim grant,2021Q4,mouse,imaging,light sheet microscopy,27,0,BIL,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,zhuang_dulac_merfish_MOE,U19 Zeng grant,2021Q1,mouse,spatial transcriptomics,merfish,0,0,BIL,0,3300000.0,0,0
0,zhuang_dulac_merfish_PAG,U19 Huang grant,2020Q2,mouse,spatial transcriptomics,merfish,0,0,BIL,0,2900000.0,0,0
0,zhuang_dulac_merfish_PAG_2,U19 Huang grant,2021Q2,mouse,spatial transcriptomics,merfish,0,0,BIL,0,2900000.0,0,0
0,zhuang_merfish_M1,multi-grant,2020Q1,mouse,spatial transcriptomics,merfish,0,0,BIL,0,600000.0,0,0


In [410]:
count_df['species'].as_list().unique()

AttributeError: 'DataFrame' object has no attribute 'as_list'

In [407]:
hex_colors[0:10]

('#F0F8FF',
 '#FAEBD7',
 '#00FFFF',
 '#7FFFD4',
 '#F0FFFF',
 '#F5F5DC',
 '#FFE4C4',
 '#000000',
 '#FFEBCD',
 '#0000FF')